In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import mysql.connector
from google.cloud import bigquery
import datetime
import db_dtypes
from plotly import graph_objects as go
import plotly.figure_factory as ff


mydb = mysql.connector.connect(
  host="ro.db.investengine.com",
  user="renat.yunisov",
  password="HtyfnHtyfn2002%",
  database="investengine"
)

In [3]:
sql_query = """
SELECT
    "1. Chose client type" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
-- WHERE
--     c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
--     AND DATE_SUB(NOW(), interval 1 week)
--  [[AND {{utm_source}}]]
--  [[AND {{utm_source}}]]
GROUP BY 1,2,3,4,5
UNION
SELECT
    "2. Signed up" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
WHERE 1=1
    -- c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
    -- AND DATE_SUB(NOW(), interval 1 week)
    AND c.state IN("NOT_COMPLETED", "COMPLETED", "APPROVED", "CLOSING", "CLOSED")
--  [[AND {{utm_source}}]]
--  [[AND {{utm_medium}}]]
GROUP BY 1,2,3,4,5
UNION
SELECT
    "3. Created portfolio" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
    INNER JOIN (
        SELECT
            g.client_id,
            COUNT(g.id) portfolio_qnt
        FROM
            investengine.portfolio_goal g
        WHERE g.manage_type != "CASH"
        GROUP BY
            g.client_id) cg ON cg.client_id = c.id
WHERE 1=1
    -- c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
    -- AND DATE_SUB(NOW(), interval 1 week)
    AND c.state IN("NOT_COMPLETED", "COMPLETED", "APPROVED", "CLOSING", "CLOSED")
--  [[AND {{utm_source}}]]
--  [[AND {{utm_medium}}]]
GROUP BY 1,2,3,4,5
UNION
SELECT
    "4. Finished registration" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
WHERE 1=1
    -- c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
    -- AND DATE_SUB(NOW(), interval 1 week)
    AND c.state IN("COMPLETED", "APPROVED", "CLOSING", "CLOSED")
--  [[AND {{utm_source}}]]
--  [[AND {{utm_medium}}]]
GROUP BY 1,2,3,4,5
UNION
SELECT
    "5. Approved" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
WHERE 1=1
    -- c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
    -- AND DATE_SUB(NOW(), interval 1 week)
    AND c.state IN("APPROVED")
--  [[AND {{utm_source}}]]
--  [[AND {{utm_medium}}]]
GROUP BY 1,2,3,4,5
UNION
SELECT
    "6. Funded" AS Step,
    type,
    DATE_FORMAT(date_joined, "%Y-%m-%d"),
    utm_source,
    utm_medium,
    count(*) AS Value
FROM
    investengine.clients_client c
    LEFT JOIN investengine.marketing_acquisitiondata on marketing_acquisitiondata.client_id = c.id
    INNER JOIN (
        SELECT
            g.client_id,
            SUM(t.credit) contributions
        FROM
            investengine.portfolio_goal g
            INNER JOIN investengine.transactions_transaction t ON t.goal_id = g.id
        WHERE
            t.trans_type IN("TOPUP", "ISA_TOPUP")
            AND t. `state` in("DONE", "CONFIRMED", "UNCONFIRMED")
            AND t.bonus IS NULL
        GROUP BY
            g.client_id) cc ON cc.client_id = c.id
-- WHERE
--     c.created BETWEEN DATE_SUB(DATE_SUB(NOW(), interval 1 week), interval 1 month)
--     AND DATE_SUB(NOW(), interval 1 week)
--  [[AND {{utm_source}}]]
--  [[AND {{utm_medium}}]]
GROUP BY 1,2,3,4,5
;
"""

In [4]:
result_dataFrame = pd.read_sql(sql_query, mydb)
result_dataFrame = result_dataFrame.rename(columns={'DATE_FORMAT(date_joined, "%Y-%m-%d")':'date'})

result_dataFrame['date'] = pd.to_datetime(result_dataFrame.date)
result_dataFrame['date_month'] = pd.to_datetime(result_dataFrame.date).dt.to_period('M').dt.to_timestamp()
result_dataFrame['date_week'] = pd.to_datetime(result_dataFrame.date).dt.to_period('W').dt.to_timestamp()
result_dataFrame['date_quarter'] = pd.PeriodIndex(result_dataFrame.date, freq='Q')

onboarding_funnel = result_dataFrame.groupby(['type', 'date_quarter', 'utm_source', 'utm_medium'], as_index=False)['Value'].agg('sum') \
                .sort_values('Value', ascending=False)

file_name = 'Onboarding data (funnel).xlsx'
 
result_dataFrame.to_excel(file_name)

onboarding_funnel['date_quarter_str'] = onboarding_funnel['date_quarter'].astype(str)
onboarding_funnel['date_source'] = onboarding_funnel['date_quarter_str'] + ' | ' + onboarding_funnel['utm_source']

onbording_funnel_chart = onboarding_funnel.groupby(['type', 'date_source'], as_index=False)['Value'].agg('sum') \
                        .sort_values(['type', 'Value'], ascending=False)

fig_funnel1 = px.bar(onbording_funnel_chart.groupby('type', as_index=False).head(10).reset_index(drop=True), 
           x='Value', 
           y='date_source', 
           color='type', 
           text_auto=True)
fig_funnel1.show()


bar_chart_funnel = onboarding_funnel.groupby(['type', 'date_quarter'], as_index=False)['Value'].agg('sum') \
                    .sort_values(['type', 'date_quarter', 'Value'], ascending=False) \
                    .groupby('type', as_index=False).head(10).reset_index(drop=True)

bar_chart_funnel['date_quarter'] = bar_chart_funnel['date_quarter'].astype(str)

figure_funnel2 = px.bar(bar_chart_funnel, 
           x='Value', 
           y='date_quarter', 
           color='type', 
           text_auto=True)
figure_funnel2.show()

/var/folders/zr/tv_3wchj6g94zkzdljg_jlnw0000gn/T/ipykernel_3207/912898090.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataFrame = pd.read_sql(sql_query, mydb)
